In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
movie_df = pd.read_csv('./data/ml-latest-small/movies.csv')
rating_df = pd.read_csv('./data/ml-latest-small/ratings.csv')
link_df = pd.read_csv('./data/ml-latest-small/links.csv')
tag_df = pd.read_csv('./data/ml-latest-small/tags.csv')

In [3]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
rating_df = rating_df.rename(columns={'movieId': 'movie_id'})
movie_df = movie_df.rename(columns={'movieId': 'movie_id'})
rating_df = rating_df.rename(columns={'userId': 'user_id'})


In [8]:
# calculate the number of ratings per movie
rating_count = rating_df.groupby('movie_id')[['rating']].count()

In [9]:
# filter for movies with more than 20 ratings and extract the index
popular_movies = rating_count[rating_count['rating']>20].index

In [19]:
# filter the ratings matrix and only keep the popular movies
df = rating_df[rating_df['movie_id'].isin(popular_movies)].copy()
df

,user_id,movie_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100803,610,148626,4.0,1493847175
100808,610,152081,4.0,1493846503
100829,610,164179,5.0,1493845631
100830,610,166528,4.0,1493879365


In [20]:
rating_df.shape, df.shape

((100836, 4), (66658, 4))

In [23]:
# need to remake user ids and movie ids since they are not sequential
user_ids = df['user_id'].unique()
user_id_map = {v:k for k,v in enumerate(user_ids)}
df['user_id'] = df['user_id'].map(user_id_map)

#similarly for the movie_id:
movie_ids = df['movie_id'].unique()
movie_id_map = {v:k for k,v in enumerate(movie_ids)}
df['movie_id'] = df['movie_id'].map(movie_id_map)
df

,user_id,movie_id,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224
3,0,3,5.0,964983815
4,0,4,5.0,964982931
...,...,...,...,...
100803,609,808,4.0,1493847175
100808,609,643,4.0,1493846503
100829,609,809,5.0,1493845631
100830,609,644,4.0,1493879365


In [24]:
# filter out unpopular movies
movies = movie_df[movie_df['movie_id'].isin(movie_ids)]

In [25]:
# redefine movie ids
movies['movie_id'] = movies['movie_id'].map(movie_id_map)
movies

/var/folders/ch/f6wpk76d1z90hnjk7lhgsmgc0000gp/T/ipykernel_35489/2301476842.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['movie_id'] = movies['movie_id'].map(movie_id_map)


,movie_id,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
930,1230,Annie Hall (1977),Comedy|Romance
931,1231,"Right Stuff, The (1983)",Drama
932,1232,Stalker (1979),Drama|Mystery|Sci-Fi
933,1233,"Boot, Das (Boat, The) (1981)",Action|Drama|War


In [13]:
from scipy.sparse import csr_matrix
R = csr_matrix((df['rating'], (df['user_id'], df['movie_id'])))

In [14]:
R.shape

(610, 1235)